
# Second-order methods

## Heun's method

Heun's method is also known as the modified Euler method. The idea here is to use the average of the
two derivatives instead of using only the derivative at the current point:
\begin{align*}
y_{j+1} & = y_{j}+ \frac{h}{2} \left( y'(t_{j}) + y'(t_{j+1})\right)
\\ & = y_{j}+ \frac {h}{2} \left( f(t_{j},y_{j})+f(t_{j+1},y_{j+1})\right)
\end{align*}
Observe that we do require to approximate $f(t_{j+1},y_{j+1})$. Therefore,
\begin{align*}
\begin{cases}
{\tilde {y}}_{j+1} &= y_i + h f(t_{j},y_{j})\\
y_{j+1} & = y_{j}+ \frac {h}{2} \left(f(t_{j},y_{j})+f(t_{j+1},{\tilde {y}}_{j+1})\right)
\end{cases}
\end{align*}
where $y_{j}$ for $j = 0, 1, 2, \ldots ,N - 1$ are approximated.

::::{tab-set}

:::{tab-item} Python Code
```python
import pandas as pd 
import numpy as np

def HeunsMethod(f, y0, a, b, h= False, N=False):
    '''
    Parameters
    ----------
    f : function
        DESCRIPTION. the ODE y'=f(t,y(x))
    y0 : float
        the initial value.
    a : float
        DESCRIPTION. a is the left side of interval [a, b]
    b : float
        DESCRIPTION. b is the right side of interval [a, b]
    h : float, optional
        DESCRIPTION. The default is False. stepsize
    N : int, optional
        DESCRIPTION. The default is False. number of points.

    Returns
    -------
    Table : dataframe
        DESCRIPTION. a summary of the algorithm output

    '''
    if N:
        h = (b-a)/(N)
    if h:
        N = int((b-a)/h)
    t = np.linspace(a, b, N+1)
    y = np.zeros(t.shape, dtype=float)
    y[0] = y0
    for j in range(N):
        z = y[j] + h*f(t[j], y[j])
        y[j+1] = y[j]+ (h/2)*(f(t[j],y[j]) + f(t[j+1],z))
        del z
    Table = pd.DataFrame({'t':t, 'y':y})
    return Table
```
:::

:::{tab-item} MATLAB Code
```MATLAB
function [Table] = HeunsMethod(f, y0, a, b, N)
%{
Parameters
----------
f : function
    DESCRIPTION. the ODE y'=f(t,y(x))
y0 : float
    the initial value.
a : float
    DESCRIPTION. a is the left side of interval [a, b]
b : float
    DESCRIPTION. b is the right side of interval [a, b]
N : int
    DESCRIPTION. Number of points.

Returns
-------
Table : dataframe
    DESCRIPTION. a summary of the algorithm output

Example:
f = @(t, y) t.*exp(-t.^2)-2.*t.*y
a = 0
b = 1
N = 10
y0 = 1
%}
h = (b-a)/(N);
t = linspace(a, b, N+1)';
y = zeros(length(t),1);
y(1) = y0;
% loop
for j=1:N
    z = y(j) + h*f(t(j), y(j));
    y(j+1) = y(j) + h*(f(t(j), y(j)) + f(t(j+1), z))/2;
end
Table = table(t,y);
end
```
:::

::::

In [1]:
import sys
sys.path.insert(0,'..')
import hd_tools as hd

Loading BokehJS ...

<font color='Blue'><b>Example</b></font>: Consider the following IVP,
\begin{align*}
\begin{cases}
y'+2ty=te^{-t^2},\quad 0 \leq t \leq 1,\\
y(0) = 1,
\end{cases}
\end{align*}
with exact solution
\begin{align*}
y\left(t \right) = \left(1 + \dfrac{t^{2}}{2}\right) e^{- t^{2}}.
\end{align*}
Use Heun's method method for solving this IVP. Also, investigate the order of convergence numerically.

<font color='Green'><b>Solution</b></font>:

In [2]:
import numpy as np
import pandas as pd
from hd_IVP_Algorithms import HeunsMethod  

# f(t, y(t)):
f = lambda t, y: t*np.exp(-t**(2))-2*t*y
(a, b) = (0, 1)
# the eact solution y(t)
y_exact = lambda t: (1+(t**2)/2)*np.exp(-t**2)
#
y0 = 1
# Table
Table = HeunsMethod(f = f, y0 = y0, a = a, b = b, N = 10)
Table['Exact'] = y_exact(Table['t'])
Table['Error'] =  np.abs(Table['Exact'] - Table['y'])
display(Table[1:].style.set_properties(subset=['Error'], **{'background-color': 'Lavender', 'color': 'Navy',
                                                'border-color': 'DarkGreen'}).format({'Error': "{:.4e}"}))

,t,y,Exact,Error
1,0.100000,0.994950,0.995000,4.9834e-05
2,0.200000,0.979860,0.980005,1.4537e-04
3,0.300000,0.954783,0.955058,2.7501e-04
4,0.400000,0.919895,0.920315,4.2042e-04
5,0.500000,0.875593,0.876151,5.5836e-04
6,0.600000,0.822595,0.823258,6.6326e-04
7,0.700000,0.762009,0.762720,7.1061e-04
8,0.800000,0.695345,0.696026,6.8072e-04
9,0.900000,0.624463,0.625026,5.6222e-04
10,1.000000,0.551465,0.551819,3.5464e-04


For the error analysis, recall that,
\begin{align*}
E_{h} = \max_{1 \leq j \leq \frac{b-a}{h}}\left| y(t_{j}) - y_{j}\right|,
\end{align*}
then,

In [3]:
Cols = ['h', 'N', 'Eh']
h = [2**(-i) for i in range(3, 12)]
Table = pd.DataFrame(np.zeros([len(h), len(Cols)], dtype = float), columns=Cols)
Table['h'] = h
Table['N'] = ((b-a)/Table['h']).astype(int)

for n in range(Table.shape[0]):
    TB = HeunsMethod(f = f, y0 = y0, a = a, b = b, h = Table['h'][n])
    Table.loc[n, 'Eh'] = np.max(np.abs(y_exact(TB['t'])[1:] - TB['y'][1:]))
        
display(Table.style.set_properties(subset=['h', 'N'],
                                   **{'background-color': 'PaleGreen', 'color': 'Black',
                                      'border-color': 'DarkGreen'}).format({'h':'{:.4e}','Eh':'{:.4e}'}))

hd.derivative_ConvergenceOrder(vecs = [Table['Eh'].values],
                            labels = ["""Heun's Method"""],
                            xlabel = r"$$i$$",
                            ylabel = r"$$\ln \left( E_{h_{i}} / E_{h_{i-1}}  \right)$$",
                            title = 'Order of Accuracy: %s' % """Heun's Method""",
                            legend_orientation = 'horizontal', ylim = [1.9, 2.25])

,h,N,Eh
0,1.2500e-01,8,1.1751e-03
1,6.2500e-02,16,2.5226e-04
2,3.1250e-02,32,5.8166e-05
3,1.5625e-02,64,1.3962e-05
4,7.8125e-03,128,3.4164e-06
5,3.9062e-03,256,8.4501e-07
6,1.9531e-03,512,2.1011e-07
7,9.7656e-04,1024,5.2386e-08
8,4.8828e-04,2048,1.3079e-08


## Trapezoidal rule

This method is similar to second-order Heun's method; however, $y_{j+1}$ is required to be approximated implicitly. This method is defined as follows
\begin{align*}
y_{j+1} = y_{j} + \frac{h}{2}\left(f(t_{j}, y_{j}) + f(t_{j+1}, y_{j+1})\right).
\end{align*}
where $y_{j+1}$ for $j = 0, 1, 2, \ldots ,N − 1$ are approximated by solving the above equation in each step.

::::{tab-set}

:::{tab-item} Python Code
```python
import pandas as pd 
import numpy as np
from sympy import symbols, solve

def TrapezoidalRule(f, y0, a, b, h= False, N=False):
    '''
    Parameters
    ----------
    f : function
        DESCRIPTION. the ODE y'=f(t,y(x))
    y0 : float
        the initial value.
    a : float
        DESCRIPTION. a is the left side of interval [a, b]
    b : float
        DESCRIPTION. b is the right side of interval [a, b]
    h : float, optional
        DESCRIPTION. The default is False. stepsize
    N : int, optional
        DESCRIPTION. The default is False. number of points.

    Returns
    -------
    Table : dataframe
        DESCRIPTION. a summary of the algorithm output

    '''
    if N:
        h = (b-a)/(N)
    if h:
        N = int((b-a)/h)
    t = np.linspace(a, b, N+1)
    y = np.zeros(t.shape, dtype=float)
    y[0] = y0
    b = symbols('b')
    for j in range(N):
        y[j+1] = solve(y[j] + 0.5*h*f(t[j], y[j]) + 0.5*h*f(t[j+1], b) - b)[0]
    Table = pd.DataFrame({'t':t, 'y':y})
    return Table
```
:::

:::{tab-item} MATLAB Code
```MATLAB
function [Table] = TrapezoidalRule(f, y0, a, b, N)
%{
Parameters
----------
f : function
    DESCRIPTION. the ODE y'=f(t,y(x))
y0 : float
    the initial value.
a : float
    DESCRIPTION. a is the left side of interval [a, b]
b : float
    DESCRIPTION. b is the right side of interval [a, b]
N : int
    DESCRIPTION. Number of points.

Returns
-------
Table : dataframe
    DESCRIPTION. a summary of the algorithm output

Example:
f = @(t, y) t.*exp(-t.^2)-2.*t.*y
a = 0
b = 1
N = 10
y0 = 1
%}
h = (b-a)/(N);
t = linspace(a, b, N+1)';
y = zeros(length(t),1);
y(1) = y0;
syms b
% loop
for j=1:N
    y(j+1) = vpasolve(y(j) + 0.5*h*f(t(j), y(j)) + 0.5*h*f(t(j+1), b) - b);
end
Table = table(t,y);
end
```
:::

::::

<font color='Blue'><b>Example</b></font>: Consider the following IVP,
\begin{align*}
\begin{cases}
y'+2ty=te^{-t^2},\quad 0 \leq t \leq 1,\\
y(0) = 1,
\end{cases}
\end{align*}
with exact solution
\begin{align*}
y\left(t \right) = \left(1 + \dfrac{t^{2}}{2}\right) e^{- t^{2}}.
\end{align*}
Use Trapezoidal rule method for solving this IVP. Also, investigate the order of convergence numerically.

<font color='Green'><b>Solution</b></font>:

In [4]:
from hd_IVP_Algorithms import TrapezoidalRule

# f(t, y(t)):
f = lambda t, y: t*np.exp(-t**(2))-2*t*y
(a, b) = (0, 1)
# the eact solution y(t)
y_exact = lambda t: (1+(t**2)/2)*np.exp(-t**2)
#
y0 = 1
# Table
Table = TrapezoidalRule(f = f, y0 = y0, a = a, b = b, N = 10)
Table['Exact'] = y_exact(Table['t'])
Table['Error'] =  np.abs(Table['Exact'] - Table['y'])
display(Table[1:].style.set_properties(subset=['Error'], **{'background-color': 'Lavender', 'color': 'Navy',
                                                'border-color': 'DarkGreen'}).format({'Error': "{:.4e}"}))

,t,y,Exact,Error
1,0.100000,0.995000,0.995000,1.6378e-07
2,0.200000,0.980008,0.980005,2.9954e-06
3,0.300000,0.955073,0.955058,1.4650e-05
4,0.400000,0.920358,0.920315,4.2785e-05
5,0.500000,0.876244,0.876151,9.3547e-05
6,0.600000,0.823427,0.823258,1.6884e-04
7,0.700000,0.762985,0.762720,2.6472e-04
8,0.800000,0.696397,0.696026,3.7148e-04
9,0.900000,0.625501,0.625026,4.7533e-04
10,1.000000,0.552380,0.551819,5.6122e-04


In [5]:
Cols = ['h', 'N', 'Eh']
h = [2**(-i) for i in range(3, 8)]
Table = pd.DataFrame(np.zeros([len(h), len(Cols)], dtype = float), columns=Cols)
Table['h'] = h
Table['N'] = ((b-a)/Table['h']).astype(int)

for n in range(Table.shape[0]):
    TB = TrapezoidalRule(f = f, y0 = y0, a = a, b = b, h = Table['h'][n])
    Table.loc[n, 'Eh'] = np.max(np.abs(y_exact(TB['t'])[1:] - TB['y'][1:]))
        
display(Table.style.set_properties(subset=['h', 'N'],
                                   **{'background-color': 'PaleGreen', 'color': 'Black',
                                      'border-color': 'DarkGreen'}).format({'h':'{:.4e}','Eh':'{:.4e}'}))

hd.derivative_ConvergenceOrder(vecs = [Table['Eh'].values],
                            labels = ['Trapezoidal rule'],
                            xlabel = r"$$i$$",
                            ylabel = r"$$\ln \left( E_{h_{i}} / E_{h_{i-1}}  \right)$$",
                            title = 'Order of Accuracy: %s' % 'Trapezoidal rule',
                            legend_orientation = 'horizontal', ylim = [1.9, 2.1])

,h,N,Eh
0,1.2500e-01,8,8.7616e-04
1,6.2500e-02,16,2.1942e-04
2,3.1250e-02,32,5.4879e-05
3,1.5625e-02,64,1.3721e-05
4,7.8125e-03,128,3.4359e-06


## Midpoint methods

The midpoint method for solving an IVP can be done explicitly or implicitly. The explicit midpoint method is given by the formula

\begin{align*}
y_{n+1}=y_{n}+hf\left(t_{n}+{\frac {h}{2}},~y_{n}+{\frac {h}{2}}f(t_{n},y_{n})\right),
\end{align*}
This process is carried forward iteratively until point $x_{n-1}$ is reached. Furthermore, the implicit midpoint method by
\begin{align*}
y_{n+1}=y_{n}+hf\left(t_{n}+{\frac {h}{2}},~{\frac {1}{2}}(y_{n}+y_{n+1})\right),
\end{align*}
where $y_{j+1}$ for $j = 0, 1, 2, \ldots ,N - 1$ are approximated by solving the above equation in each step.

::::{tab-set}

:::{tab-item} Python Code
```python
import pandas as pd 
import numpy as np
from sympy import symbols, solve

def ExplicitMidpointMethod(f, y0, a, b, h= False, N=False):
    '''
    Parameters
    ----------
    f : function
        DESCRIPTION. the ODE y'=f(t,y(x))
    y0 : float
        the initial value.
    a : float
        DESCRIPTION. a is the left side of interval [a, b]
    b : float
        DESCRIPTION. b is the right side of interval [a, b]
    h : float, optional
        DESCRIPTION. The default is False. stepsize
    N : int, optional
        DESCRIPTION. The default is False. number of points.

    Returns
    -------
    Table : dataframe
        DESCRIPTION. a summary of the algorithm output
    '''
    if N:
        h = (b-a)/(N)
    if h:
        N = int((b-a)/h)
    t = np.linspace(a, b, N+1)
    y = np.zeros(t.shape, dtype=float)
    y[0] = y0
    for j in range(N):
        z = f(t[j], y[j])
        y[j+1] = y[j+1] = y[j] + h*f(t[j] + h/2, y[j] + (h/2)*z)
        del z
    Table = pd.DataFrame({'t':t, 'y':y})
    return Table

def ImplicitMidpointMethod(f, y0, a, b, h= False, N=False):
    '''
    Parameters
    ----------
    f : function
        DESCRIPTION. the ODE y'=f(t,y(x))
    y0 : float
        the initial value.
    a : float
        DESCRIPTION. a is the left side of interval [a, b]
    b : float
        DESCRIPTION. b is the right side of interval [a, b]
    h : float, optional
        DESCRIPTION. The default is False. stepsize
    N : int, optional
        DESCRIPTION. The default is False. number of points.

    Returns
    -------
    Table : dataframe
        DESCRIPTION. a summary of the algorithm output
    '''
    if N:
        h = (b-a)/(N)
    if h:
        N = int((b-a)/h)
    t = np.linspace(a, b, N+1)
    y = np.zeros(t.shape, dtype=float)
    y[0] = y0
    b = symbols('b')
    for j in range(N):
        y[j+1] = solve(y[j] + h*f(t[j] + h/2, 0.5*(y[j] + b)) - b)[0]
    Table = pd.DataFrame({'t':t, 'y':y})
    return Table
```
:::

:::{tab-item} MATLAB Code
```MATLAB
function [Table] = ExplicitMidpointMethod(f, y0, a, b, N)
%{
Parameters
----------
f : function
    DESCRIPTION. the ODE y'=f(t,y(x))
y0 : float
    the initial value.
a : float
    DESCRIPTION. a is the left side of interval [a, b]
b : float
    DESCRIPTION. b is the right side of interval [a, b]
N : int
    DESCRIPTION. Number of points.

Returns
-------
Table : dataframe
    DESCRIPTION. a summary of the algorithm output

Example:
f = @(t, y) t.*exp(-t.^2)-2.*t.*y
a = 0
b = 1
N = 10
y0 = 1
%}
h = (b-a)/(N);
t = linspace(a, b, N+1)';
y = zeros(length(t),1);
y(1) = y0;
% loop
for j=1:N
    z = h*f(t(j), y(j));
    y(j+1) = y(j) + h*f(t(j) + h/2, y(j) + (h/2)*z);
end
Table = table(t,y);
end

function [Table] = ImplicitMidpointMethod(f, y0, a, b, N)
%{
Parameters
----------
f : function
    DESCRIPTION. the ODE y'=f(t,y(x))
y0 : float
    the initial value.
a : float
    DESCRIPTION. a is the left side of interval [a, b]
b : float
    DESCRIPTION. b is the right side of interval [a, b]
N : int
    DESCRIPTION. Number of points.

Returns
-------
Table : dataframe
    DESCRIPTION. a summary of the algorithm output

Example:
f = @(t, y) t.*exp(-t.^2)-2.*t.*y
a = 0
b = 1
N = 10
y0 = 1
%}
h = (b-a)/(N);
t = linspace(a, b, N+1)';
y = zeros(length(t),1);
y(1) = y0;
syms b
% loop
for j=1:N
    y(j+1) = vpasolve(y(j) + h*f(t(j) + h/2, 0.5*(y(j) + b)) - b);
end
Table = table(t,y);
end
```
:::

::::

<font color='Blue'><b>Example</b></font>: Consider the following IVP,
\begin{align*}
\begin{cases}
y'+2ty=te^{-t^2},\quad 0 \leq t \leq 1,\\
y(0) = 1,
\end{cases}
\end{align*}
with exact solution
\begin{align*}
y\left(t \right) = \left(1 + \dfrac{t^{2}}{2}\right) e^{- t^{2}}.
\end{align*}
Use Midpoint methods for solving this IVP. Also, investigate the order of convergence numerically.

<font color='Green'><b>Solution</b></font>:


In [6]:
from hd_IVP_Algorithms import ExplicitMidpointMethod, ImplicitMidpointMethod

# f(t, y(t)):
f = lambda t, y: t*np.exp(-t**(2))-2*t*y
(a, b) = (0, 1)
# the eact solution y(t)
y_exact = lambda t: (1+(t**2)/2)*np.exp(-t**2)
#
y0 = 1

# Table
Table = ExplicitMidpointMethod(f = f, y0 = y0, a = a, b = b, N = 10).rename(columns = {'y':'y (Explicit)'})
Table['Exact'] = y_exact(Table['t'])
Table['Error (Explicit)'] =  np.abs(Table['Exact'] - Table['y (Explicit)'])
Table['Error (Explicit)'] =  np.abs(Table['Exact'] - Table['y (Explicit)'])

Temp = ImplicitMidpointMethod(f = f, y0 = y0, a = a, b = b, N = 10).rename(columns = {'y':'y (Implicit)'})
Temp['Exact'] = y_exact(Table['t'])
Temp['Error (Implicit)'] =  np.abs(Temp['Exact'] - Temp['y (Implicit)'])
Temp['Error (Implicit)'] =  np.abs(Temp['Exact'] - Temp['y (Implicit)'])
Table = Table.merge(Temp, on = ['t', 'Exact'])

Table = Table.reindex(sorted(Table.columns), axis=1)
Cols = [x for x in Table.columns if 'Error' in x]
display(Table[1:].style.set_properties(subset= Cols,**{'background-color': 'Lavender', 'color': 'Navy',
                                                       'border-color': 'DarkGreen'})\
       .format(dict(zip(Cols, len(Cols)*['{:.4e}']))))

,Error (Explicit),Error (Implicit),Exact,t,y (Explicit),y (Implicit)
1,1.2567e-05,1.2370e-05,0.995000,0.100000,0.994988,0.995012
2,5.1080e-05,4.7516e-05,0.980005,0.200000,0.979954,0.980053
3,1.1676e-04,9.9718e-05,0.955058,0.300000,0.954941,0.955158
4,2.0940e-04,1.6006e-04,0.920315,0.400000,0.920106,0.920475
5,3.2511e-04,2.1740e-04,0.876151,0.500000,0.875826,0.876368
6,4.5452e-04,2.5975e-04,0.823258,0.600000,0.822804,0.823518
7,5.8244e-04,2.7597e-04,0.762720,0.700000,0.762137,0.762996
8,6.8933e-04,2.5759e-04,0.696026,0.800000,0.695337,0.696284
9,7.5452e-04,2.0033e-04,0.625026,0.900000,0.624271,0.625226
10,7.6042e-04,1.0516e-04,0.551819,1.000000,0.551059,0.551924


In [17]:
Cols = ['h', 'N']
h = [2**(-i) for i in range(3, 8)]
Table = pd.DataFrame(np.zeros([len(h), len(Cols)], dtype = float), columns=Cols)
Table['h'] = h
Table['N'] = ((b-a)/Table['h']).astype(int)

for n in range(Table.shape[0]):
    TB = ExplicitMidpointMethod(f = f, y0 = y0, a = a, b = b, h = Table['h'][n])
    Table.loc[n, 'Eh (Explicit)'] = np.max(np.abs(y_exact(TB['t'])[1:] - TB['y'][1:]))
    
    TB = ImplicitMidpointMethod(f = f, y0 = y0, a = a, b = b, h = Table['h'][n])
    Table.loc[n, 'Eh (Implicit)'] = np.max(np.abs(y_exact(TB['t'])[1:] - TB['y'][1:]))
       
Cols = [x for x in Table.columns if 'Eh' in x]
display(Table.style.set_properties(subset=['h', 'N'], **{'background-color': 'PaleGreen', 'color': 'Black',
       'border-color': 'DarkGreen'}).format(dict(zip(Cols, len(Cols)*["{:.4e}"]))))

hd.derivative_ConvergenceOrder(vecs = [Table['Eh (Explicit)'].values, Table['Eh (Implicit)'].values],
                            labels = ['Explicit Midpoint Method', 'Implicit Midpoint Method'],
                            xlabel = r"$$i$$",
                            ylabel = r"$$\ln \left( E_{h_{i}} / E_{h_{i-1}}  \right)$$",
                            title = 'Order of Accuracy: %s' % 'Trapezoidal rule',
                            legend_orientation = 'horizontal', ylim = [1.9, 2.25])

,h,N,Eh (Explicit),Eh (Implicit)
0,0.125000,8,1.2295e-03,4.2623e-04
1,0.062500,16,2.8534e-04,1.0722e-04
2,0.031250,32,6.8695e-05,2.6747e-05
3,0.015625,64,1.6854e-05,6.6921e-06
4,0.007812,128,4.1743e-06,1.6720e-06


***
**References:**
1. Allaire, Grégoire, et al. Numerical linear algebra. Vol. 55. New York: Springer, 2008.
1. Burden, Richard L., and J. Douglas Faires. "Numerical analysis 8th ed." Thomson Brooks/Cole (2005).
1. Atkinson, Kendall E. An introduction to numerical analysis. John wiley & sons, 2008.
1. Khoury, Richard, and Douglas Wilhelm Harder. Numerical methods and modelling for engineering. Springer, 2016.
1. Zarowski, Christopher J. An introduction to numerical analysis for electrical and computer engineers. John Wiley & Sons, 2004.
1. [Heun's method](https://en.wikipedia.org/wiki/Heun%27s_method)
***